### OCI Data Science - Useful Tips
Everything stored in the <span style="background-color: #d5d8dc ">/home/datascience</span> folder is now stored on your block volume drive. The <span style="background-color: #d5d8dc ">ads-examples</span> folder has moved outside of your working space. Notebook examples are now accessible through a Launcher tab "Notebook Examples" button.
<details>
<summary><font size="2">1. Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">2. OCI Configuration and Key Files Set Up</font></summary><p>Follow the instructions in the getting-started notebook. That notebook is accessible via the "Getting Started" Launcher tab button.</p>
</details>
<details>
<summary><font size="2">3. Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">4. Typical Cell Imports and Settings</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import MLData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">5. Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [8]:
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import logging
import os
import cx_Oracle
#logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
#from ads.dataset.factory import DatasetFactory
#from sqlalchemy import create_engine

In [3]:
os.environ['TNS_ADMIN'] = "/home/datascience/ADB"
connection = cx_Oracle.connect('hackathon', 'bCfrCC0981267', 'adw_low')

In [246]:
import matplotlib.pyplot as plt

In [247]:
plt.close('all')

In [4]:
# Modelo de perfil 
PJPERFILPJ = pd.read_sql("SELECT TKTATC_DS_SG_PFPJMISTO, COUNT(*) AS QTDE FROM EVE_MODELO_PERFILPJ_ATC_AQUISICAO_HCKT GROUP BY TKTATC_DS_SG_PFPJMISTO", con=connection) 

In [5]:
PERFILPJ

NameError: name 'PERFILPJ' is not defined

In [389]:
TRS_RESTAURANTES_LISTA = pd.read_sql("SELECT TAUT_DS_ENTRYMODE, COUNT(*) AS QTDE FROM EVE_AUTHORIZATIONS_MANU_HCKT WHERE TAUT_CD_MCC IN (5499, 5812, 5813, 5814) GROUP BY TAUT_DS_ENTRYMODE", con=connection) 

In [393]:
TRS_RESTAURANTES_LISTA.head()

,TAUT_DS_ENTRYMODE,QTDE
0,053,381
1,0500,2
2,0710,19138
3,012,769
4,TARJA,508


In [474]:
# Lista de restaurantes no que receberam os Cartões Carrefour
TRANSACAO_RESTAURANTES_LISTA = pd.read_sql("SELECT DISTINCT TAUT_DS_ESTABELECIMENTO, COUNT(*) AS QTDE, SUM( TAUT_VL_AUTORIZACAO ) AS VL_TOTAL FROM EVE_AUTHORIZATIONS_MANU_HCKT WHERE TAUT_CD_MCC IN (5499, 5812, 5813, 5814) AND TAUT_DS_ENTRYMODE IN( 'CHIP','TARJA','051','0510','0520','0210', 'CONTACTLESS','0700','071','0710','0720') AND TAUT_ST_TRANSACAO = 'APROVADA' GROUP BY TAUT_DS_ESTABELECIMENTO ORDER BY VL_TOTAL DESC", con=connection) 

In [475]:
TRANSACAO_RESTAURANTES_LISTA.head()

,TAUT_DS_ESTABELECIMENTO,QTDE,VL_TOTAL
0,SWIFT,7408,803412.05
1,MC DONALDS,20772,728844.55
2,BOMFRIOS ALIMENTOS,2630,603250.49
3,HABIB S,10241,355862.24
4,SACLAO VILA DAS MERCES,4451,352435.31


In [ ]:
# Perfil dos compradores no Atacadão
PERFILPJ_PJS = pd.read_sql("SELECT CPF_CRIP, tktatc_ds_sg_pfpjmisto as TP_CLIENTE, tbmd_cd_posicao AS ANOMES, ROW_NUMBER() OVER( PARTITION BY CPF_CRIP ORDER BY tbmd_cd_posicao DESC )  AS RN FROM EVE_MODELO_PERFILPJ_ATACADAO_MANU_HCKT WHERE tktatc_ds_sg_pfpjmisto <> 'PF'", con=connection)

In [271]:
# Transação em restaurantes onde ocorreram os maiores gastos mensais
ATACAD_RESTAURANTES = pd.read_sql("SELECT SUBSTR( TAUT_DS_TIMESTAMP, 1, 6 ) AS ANOMES, TAUT_DS_ESTABELECIMENTO, SUM( TAUT_VL_AUTORIZACAO ) AS TOT_GASTO_MENSAL, COUNT(*) AS QT_TRANSACAO,  SUM( TAUT_VL_AUTORIZACAO ) /COUNT(*) AS TKT_MEDIO   FROM EVE_AUTHORIZATIONS_MANU_HCKT WHERE TAUT_CD_MCC IN (5499, 5812, 5813, 5814) AND TAUT_ST_TRANSACAO = 'APROVADA' GROUP BY SUBSTR( TAUT_DS_TIMESTAMP, 1, 6 ), TAUT_DS_ESTABELECIMENTO ORDER BY ANOMES, TKT_MEDIO DESC", con=connection) 

In [262]:
ATACAD_RESTAURANTES.head(5)

,ANOMES,TAUT_DS_ESTABELECIMENTO,TOT_GASTO_MENSAL,QT_TRANSACAO,TKT_MEDIO
0,202001,PAG*IvanNobregaDe,25000.00,1,25000.00
1,202001,PAG*AndreRodrigue,19276.80,1,19276.80
2,202002,PAG*BrinquedosBar,18828.28,1,18828.28
3,202001,parc=112MERCADOPAGO *DENI,18461.93,1,18461.93
4,202001,PAG*BOLOSESABORES,17710.56,1,17710.56


In [269]:
# Compras no Atacadão
EVE_BUFFER_ATC_MANU_HCKT_T10 = pd.read_sql("SELECT SUBSTR( DATE_REF, 1, 6 ) AS ANOMES, CPF_CRIP, SUM( ICPA_VL_TOTALITEM ) AS TOTAL, COUNT(*) AS QTDE_TRANSACOES, SUM( ICPA_VL_TOTALITEM ) / COUNT(*) AS TKT_MEDIO  FROM EVE_BUFFER_ATC_MANU_HCKT GROUP BY SUBSTR( DATE_REF, 1, 6 ), CPF_CRIP ORDER BY TKT_MEDIO, ANOMES", con=connection)

In [272]:
EVE_BUFFER_ATC_MANU_HCKT_T10.head(10)

,ANOMES,CPF_CRIP,TOTAL,QTDE_TRANSACOES,TKT_MEDIO
0,202005,E40C734C9B1,38400.00,3,12800.000000
1,202002,E40C734C9B1,22000.00,2,11000.000000
2,202003,9612E1BFA50,9780.00,1,9780.000000
3,202007,70979DBE06C,9525.60,1,9525.600000
4,202008,8E976FF6982,27867.37,3,9289.123333
5,202004,3D4855BC3B9,9150.96,1,9150.960000
6,202008,C729485155B,9130.62,1,9130.620000
7,202008,4666B07AFC6,8882.20,1,8882.200000
8,202006,0A36D265803,17232.00,2,8616.000000
9,202008,C9EA9F235C8,8616.00,1,8616.000000


In [476]:
# LISTA DOCUMENTOS COMPRAS ATACADAO
LISTA_DOCUMENTOS = pd.read_sql("SELECT CPF_CRIP, SUM( ICPA_VL_TOTALITEM ) AS TOTAL FROM EVE_BUFFER_ATC_MANU_HCKT GROUP BY CPF_CRIP ORDER BY TOTAL", con=connection)

In [221]:
# Perfil dos compradores no Atacadão
PERFILPJ_PJS = pd.read_sql("SELECT CPF_CRIP, tktatc_ds_sg_pfpjmisto as TP_CLIENTE, tbmd_cd_posicao AS ANOMES, ROW_NUMBER() OVER( PARTITION BY CPF_CRIP ORDER BY tbmd_cd_posicao DESC )  AS RN FROM EVE_MODELO_PERFILPJ_ATACADAO_MANU_HCKT WHERE tktatc_ds_sg_pfpjmisto <> 'PF'", con=connection)

In [487]:
PERFILPJ_PJS[PERFILPJ_PJS.RN == 1].head()

,CPF_CRIP,TP_CLIENTE,ANOMES,RN
0,000058CED0B,PJ diversos,202010,1
2,0000B189B1D,PJ abastecimento,202010,1
4,0000BDE9B98,PJ diversos,202010,1
6,0001105BCCB,Misto,202010,1
8,00011ACC5C8,PJ diversos,202010,1


In [404]:
COMPRA_PERFIL_PJ = pd.merge(left=EVE_BUFFER_ATC_MANU_HCKT_T10, right=PERFILPJ_PJS[PERFILPJ_PJS.RN == 1].drop(columns = ['RN', 'ANOMES']), left_on='CPF_CRIP', right_on='CPF_CRIP')

In [488]:
COMPRA_PERFIL_PJ.head()

,ANOMES,CPF_CRIP,TOTAL,QTDE_TRANSACOES,TKT_MEDIO,TP_CLIENTE
0,202005,E40C734C9B1,38400.0,3,12800.0,PJ abastecimento
1,202002,E40C734C9B1,22000.0,2,11000.0,PJ abastecimento
2,202003,E40C734C9B1,2970.0,1,2970.0,PJ abastecimento
3,202003,9612E1BFA50,9780.0,1,9780.0,PJ abastecimento
4,202007,70979DBE06C,9525.6,1,9525.6,PJ abastecimento


In [226]:
# Tabela de teste
CPF_NOME_FAKE = pd.DataFrame({'CPF_CRIP':['800654B00FE', '5A1DF5799A6', '021ED4749B0','7C494ED1306','1A9A4526483','F51CD5B4411','6C5F6504E4B','6286D37E37D'], 'NM_ESTABELECIMENTO':['BELISCAO LANCHES','BAR PONTO DO CAFE','RIORY SUSHI','NELICE DA SILVA ME','LAURI TITTONS LANCHES LTD','FAZENDA MINEIRA','XODO DE REALENGO','EMPORIO LITORAL']})

In [489]:
CPF_NOME_FAKE.head()

,CPF_CRIP,NM_ESTABELECIMENTO
0,800654B00FE,BELISCAO LANCHES
1,5A1DF5799A6,BAR PONTO DO CAFE
2,021ED4749B0,RIORY SUSHI
3,7C494ED1306,NELICE DA SILVA ME
4,1A9A4526483,LAURI TITTONS LANCHES LTD


In [457]:
X=TRANSACAO_RESTAURANTES_LISTA[['TAUT_DS_ESTABELECIMENTO']].drop_duplicates()

In [458]:
X.head()

,TAUT_DS_ESTABELECIMENTO
0,MC DONALDS
1,SUBWAY
2,HABIB S
3,ZP *IFOODPOS
4,HABIBS


In [477]:
Y=LISTA_DOCUMENTOS[['CPF_CRIP']].drop_duplicates()

In [478]:
Y.head()

,CPF_CRIP
0,F21E1BD357C
1,9A2D4CF6DDC
2,AAC8DDC649D
3,18CA77B463D
4,F4A2B79E4DA


In [479]:
CPF_NOME_FAKE3 = pd.concat((X, Y), axis=1).dropna()

In [473]:
CPF_NOME_FAKE3.head()

,TAUT_DS_ESTABELECIMENTO,CPF_CRIP
0,MC DONALDS,E40C734C9B1
2,HABIB S,9612E1BFA50
3,ZP *IFOODPOS,70979DBE06C
4,HABIBS,8E976FF6982
5,MERCATO EXPRESS HOLDIN,3D4855BC3B9


In [482]:
COMPRA_PERFIL_PJ_tot = pd.merge(left=COMPRA_PERFIL_PJ, right=CPF_NOME_FAKE3, left_on='CPF_CRIP', right_on='CPF_CRIP')

In [483]:
COMPRA_PERFIL_PJ_tot.head()

,ANOMES,CPF_CRIP,TOTAL,QTDE_TRANSACOES,TKT_MEDIO,TP_CLIENTE,TAUT_DS_ESTABELECIMENTO
0,202005,E40C734C9B1,38400.0,3,12800.0,PJ abastecimento,PAG*SueliPereiraDe
1,202002,E40C734C9B1,22000.0,2,11000.0,PJ abastecimento,PAG*SueliPereiraDe
2,202003,E40C734C9B1,2970.0,1,2970.0,PJ abastecimento,PAG*SueliPereiraDe
3,202003,9612E1BFA50,9780.0,1,9780.0,PJ abastecimento,MINIMERCADO E ACO
4,202007,70979DBE06C,9525.6,1,9525.6,PJ abastecimento,PAG*MEDIANEIRA


In [469]:
ATACAD_RESTAURANTES.head()

,ANOMES,TAUT_DS_ESTABELECIMENTO,TOT_GASTO_MENSAL,QT_TRANSACAO,TKT_MEDIO
0,202001,PAG*IvanNobregaDe,25000.00,1,25000.00
1,202001,PAG*AndreRodrigue,19276.80,1,19276.80
2,202001,parc=112MERCADOPAGO *DENI,18461.93,1,18461.93
3,202001,PAG*BOLOSESABORES,17710.56,1,17710.56
4,202001,SUMUP *JR,16000.00,1,16000.00


In [484]:
CLIENTES_COMPRA_PERFIL_PJ = pd.merge(left=COMPRA_PERFIL_PJ_tot, right=ATACAD_RESTAURANTES, left_on=['TAUT_DS_ESTABELECIMENTO', 'ANOMES'], right_on=['TAUT_DS_ESTABELECIMENTO','ANOMES'])

In [485]:
CLIENTES_COMPRA_PERFIL_PJ.head()

,ANOMES,CPF_CRIP,TOTAL,QTDE_TRANSACOES,TKT_MEDIO_x,TP_CLIENTE,TAUT_DS_ESTABELECIMENTO,TOT_GASTO_MENSAL,QT_TRANSACAO,TKT_MEDIO_y
0,202002,E40C734C9B1,22000.00,2,11000.00000,PJ abastecimento,PAG*SueliPereiraDe,5.0,1,5.0
1,202005,70979DBE06C,12629.76,2,6314.88000,PJ abastecimento,PAG*MEDIANEIRA,6.0,1,6.0
2,202002,3D4855BC3B9,2682.00,1,2682.00000,PJ abastecimento,XANDY RAFA LANCHES E P,8.5,1,8.5
3,202006,C729485155B,1202.12,21,57.24381,PJ abastecimento,VALDEIR DOMINGOS DA CRU,5.0,1,5.0
4,202003,4666B07AFC6,1245.00,1,1245.00000,PJ abastecimento,MERCADOPAGO *DANILOJA,6.0,1,6.0


In [486]:
CLIENTES_COMPRA_PERFIL_PJ.sort_values(by = ['TKT_MEDIO_y', 'TKT_MEDIO_x', 'QT_TRANSACAO'], ascending = False).head(10)

,ANOMES,CPF_CRIP,TOTAL,QTDE_TRANSACOES,TKT_MEDIO_x,TP_CLIENTE,TAUT_DS_ESTABELECIMENTO,TOT_GASTO_MENSAL,QT_TRANSACAO,TKT_MEDIO_y
193803,202001,41F691F6508,469.88,48,9.789167,Misto,PAG*IvanNobregaDe,25000.00,1,25000.00
31525,202002,8DA1EBFBC81,349.10,31,11.261290,PJ diversos,PAG*BrinquedosBar,18828.28,1,18828.28
7792,202002,6D23F65B824,732.22,120,6.101833,Misto,PAG*LillianeDeSouza,15060.00,1,15060.00
199511,202003,925D054CEA8,424.95,54,7.869444,Misto,parc=111MERCADOPAGO *VITO,13909.55,1,13909.55
194334,202009,3505979BF92,188.90,17,11.111765,Misto,PAG*DenisZangrando,13000.00,1,13000.00
207256,202003,392388A325C,52.38,5,10.476000,PJ abastecimento,PAG*MariaRejaneDeLima,12048.00,1,12048.00
102533,202001,E9CFBE380DA,180.99,10,18.099000,PJ diversos,PAG*SatyeMaroquio,11717.00,1,11717.00
107398,202005,CEF2E249DC3,138.88,10,13.888000,Misto,MERCADOPAGO *JULIANAR,11072.90,1,11072.90
121560,202007,3F4CA51BE77,14.58,2,7.290000,PJ abastecimento,PAG*MOMMYCAKE,11000.00,1,11000.00
121559,202006,3F4CA51BE77,65.45,5,13.090000,PJ abastecimento,PAG*MOMMYCAKE,10300.00,1,10300.00


In [420]:
CLIENTES_COMPRA_PERFIL_PJ.sort_values(by = ['TKT_MEDIO_x', 'TKT_MEDIO_y','QT_TRANSACAO'], ascending = False).head(10)

,ANOMES,CPF_CRIP,TOTAL,QTDE_TRANSACOES,TKT_MEDIO_x,TP_CLIENTE,TAUT_DS_ESTABELECIMENTO,TOT_GASTO_MENSAL,QT_TRANSACAO,TKT_MEDIO_y
0,202002,E40C734C9B1,22000.00,2,11000.000000,PJ abastecimento,ARMANDO S DA ALVORADA,24.00,1,24.000000
1,202007,70979DBE06C,9525.60,1,9525.600000,PJ abastecimento,VAREJAO PRESOTTO,574.05,15,38.270000
4,202008,8E976FF6982,27867.37,3,9289.123333,PJ abastecimento,BK - FILIAL SHOPPING M,567.60,14,40.542857
5,202004,3D4855BC3B9,9150.96,1,9150.960000,PJ abastecimento,NONA MANIA PIZZARIA,66.90,1,66.900000
6,202008,C729485155B,9130.62,1,9130.620000,PJ abastecimento,XODO DE REALENGO,675.15,17,39.714706
16,202006,0A36D265803,17232.00,2,8616.000000,PJ abastecimento,PAG*EsquinaChic,37.50,2,18.750000
18,202003,2982143D24E,8415.00,1,8415.000000,PJ abastecimento,RENDE MAIS,471.40,16,29.462500
26,202005,2512525786D,8300.00,1,8300.000000,PJ abastecimento,TO NA ESQUINA BAR,140.13,2,70.065000
28,202002,A648AA9BE60,8278.40,1,8278.400000,PJ abastecimento,PONTUAL LANCHES,56.00,3,18.666667
31,202002,B238E3279B7,8278.40,1,8278.400000,PJ abastecimento,A LEITERIA,81.50,7,11.642857


In [419]:
CLIENTES_COMPRA_PERFIL_PJ.sort_values(by = ['QT_TRANSACAO', 'TKT_MEDIO_y', 'TKT_MEDIO_x'], ascending = False).head(20)

,ANOMES,CPF_CRIP,TOTAL,QTDE_TRANSACOES,TKT_MEDIO_x,TP_CLIENTE,TAUT_DS_ESTABELECIMENTO,TOT_GASTO_MENSAL,QT_TRANSACAO,TKT_MEDIO_y
195904,202001,6EB36B45022,228.46,13,17.573846,Misto,BOBS,42098.76,1715,24.547382
248743,202006,C0417B4CCE1,32.92,3,10.973333,Misto,MC DONALDS,64060.63,1638,39.109054
195907,202002,6EB36B45022,399.78,42,9.518571,Misto,BOBS,36910.24,1520,24.283053
248742,202005,C0417B4CCE1,103.32,7,14.760000,Misto,MC DONALDS,53655.03,1336,40.160951
195905,202009,6EB36B45022,398.09,32,12.440313,Misto,BOBS,27321.25,1081,25.274052
139734,202009,BAEA02A07F8,387.60,17,22.800000,PJ abastecimento,IFOOD,47718.51,1061,44.975033
280770,202003,A5A003E1B10,5.98,1,5.980000,PJ diversos,HABIBS,31928.11,935,34.147711
195910,202008,6EB36B45022,349.79,54,6.477593,Misto,BOBS,22196.41,912,24.338169
195906,202003,6EB36B45022,636.69,66,9.646818,Misto,BOBS,20505.25,912,22.483827
280768,202009,A5A003E1B10,13.50,1,13.500000,PJ diversos,HABIBS,29220.28,897,32.575563


In [ ]:
series = pd.

In [48]:
CHAVES = pd.read_sql("SELECT * FROM EVE_CHAVES_MANU_HCKT WHERE ROWNUM < 10", con=connection) 

In [50]:
CHAVES

,NUMBERX_CRIPT,CACCSERNO_CRIPT,CPF_CRIP,CD_EMPRESA
0,58C52152A9E,A9AD5F,F787DAB536B,1
1,975A3CD6733,036AD3,224310B7CB8,1
2,C3DF462A360,F46A75,BB89F29A8CC,1
3,2B8E3D6F296,D79C86,C1D75AE10DE,1
4,49CE2A5D395,7812D1,17871BBE1D5,1
5,3D88C9B05CF,1ADB70,0462C72597A,1
6,363EE5F03C7,B83BC9,E93E4756CF1,1
7,BD89579A5BD,90EBA6,54C42FF0134,1
8,A43A02AD7AC,46CC59,18664FF1A43,1


In [54]:
TRANSACOES = pd.read_sql("SELECT * FROM EVE_AUTHORIZATIONS_MANU_HCKT WHERE ROWNUM <10", con=connection) 

In [56]:
TRANSACOES.head()

,NUMBERX_CRIPT,TAUT_DH_TRANSACAO,TAUT_DS_TIMESTAMP,TAUT_SG_FORMATO,TAUT_DS_CATEGORIA,TAUT_DS_ENTRYMODE,TAUT_DS_NATUREZA,TAUT_ST_TRANSACAO,TAUT_CD_ADQUIRENTE,TAUT_CD_RESPOSTA,TAUT_CD_MOTRESPOSTA,TAUT_CD_MCC,TAUT_QT_PARCELAS,TAUT_CD_ESTABELECIMENTO,TAUT_DS_ESTABELECIMENTO,TAUT_DS_CIDADEESTAB,TAUT_VL_AUTORIZACAO,TAUT_CD_EMPRESA,TAUT_DT_REFAUT
0,0CDC0FB4DEF,2020-01-06 23:23:22,20200106222322,M,OFF-US,051,COMPRA A VISTA,APROVADA,012088,00,0,5411,NaN,010081867800002,ZAFFARI HIGIENOP,PORTO ALEGRE,53.42,1,20200106
1,425296CEC04,2020-01-07 10:49:53,20200107094953,O,ON-US,0100,ZPAGAMENTO DE FATURA,BASE 2,622152,00,5100,5411,NaN,2003002736,273 - FOR - FORTALEZA,FORTALEZA,1873.30,1,20200106
2,425296CEC04,2020-01-07 10:49:10,20200107094910,O,ON-US,0110,ZCONSULTA DE FATURA,APROVADA,622152,00,5005,5411,NaN,2003002736,273 - FOR - FORTALEZA,FORTALEZA,0.00,1,20200106
3,425296CEC04,2020-01-07 10:49:46,20200107094946,O,ON-US,0100,ZPAGAMENTO DE FATURA,APROVADA,622152,00,0,5411,NaN,2003002736,273 - FOR - FORTALEZA,FORTALEZA,1873.30,1,20200106
4,E9EA448C249,2020-01-07 01:30:36,20200107003036,O,ON-US,0000,COMPRA PARCELADA,OUTRAS MSG,622152,00,0,5411,12.0,2003002421,Carrefour.com,CAJAMAR,1220.00,1,20200106


In [65]:
df3 = pd.merge(ATACAD2,CHAVES,how='inner', on=['NUMBERX_CRIPT'])